# Multi-agent Collaboration for Data Analytics with Cost-efficiency Strategy

## Preparation Stage

In [ ]:
%pip install \
"python-dotenv==1.0.0" \
"openai==1.6.1" \
"ag2==0.2.20" \
"pandas==2.0.3" \
"matplotlib==3.8.0" \
"seaborn==0.13.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Su

In [ ]:
%pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.12
albumentations                   1.4.11
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.7.22.0.34.13
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                

In [ ]:
%%writefile ./.env
OPENAI_API_KEY=sk-GUktlQ4YGz8DdZ1D5f02A83836444b4dAdC7D5F9BcFeFf6d
# replace https://api.openai.com/v1 with API transit address
OPENAI_BASE_URL=https://pro.aiskt.com/v1

Writing ./.env


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

def get_openai_api_key():
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_API_KEY")

def get_openai_base_url():
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_BASE_URL")

# assign corresponding value to api_key before invoking OpenAI(), once setup here, all the following calling from
# other frameworks like LlamaIndex and Trulens will inherit and don't need to config for the same
import openai
openai.api_key = get_openai_api_key()
openai.base_url = get_openai_base_url()
#OPENAI_API_KEY = get_openai_api_key()
#OPENAI_BASE_URL = get_openai_base_url()

## Multi-agent Collaboration

In [ ]:
config_list_gpt4o = {
    "config_list": [{"api_key": get_openai_api_key(), "base_url": get_openai_base_url(), "model": "gpt-4o",}],
}
config_list_gpt35turbo = {
    "config_list": [{"api_key": get_openai_api_key(), "base_url": get_openai_base_url(), "model": "gpt-3.5-turbo"}],
}

llm_config_assistant_coder = llm_config_assistant_critic = llm_config_groupchatmanager = config_list_gpt35turbo
# assign the critic assistannt with a more advanced foundation model
llm_config_assistant_critic = config_list_gpt4o

In [ ]:
import autogen

user_proxy = autogen.UserProxyAgent(
    name="HumanAdmin",
    system_message="You're a human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "./",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

coder = autogen.AssistantAgent(
    name="Coder",
    system_message="Coder, you're a senior engineer.",
    llm_config=llm_config_assistant_coder,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic, you're a helpful assistant highly skilled in evaluating the quality of a given visualization \
    code by providing a score from 1 (worst) - 10 (best) while providing clear rationale. You must consider visualization \
    best practices for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions: \
    - Bugs (bugs): Are there bugs, logic errors, syntax error or typos? Are there any reasons why the code may fail to \
    compile? How should it be fixed? If any bug exists, the bug score must be less than 5. \
    - Data transformation (transformation): Is the data transformed appropriately for the visualization type? E.g., \
    is the dataset appropriated filtered, aggregated, or grouped if needed? If a date field is used, is the date field \
    first converted to a date object, etc.? \
    - Goal compliance (compliance): How well the code meets the specified visualization goals? \
    - Visualization type (type): Considering best practices, is the visualization type appropriate for the data and intent? \
    Is there a visualization type that would be more effective in conveying insights? If a different visualization type \
    is more appropriate, the score must be less than 5. \
    - Data encoding (encoding): Is the data encoded appropriately for the visualization type? \
    - Aesthetics (aesthetics): Are the aesthetics of the visualization appropriate for the visualization type and the data? \
    You must provide a score for each of the above dimensions: \
    {bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0} \
    Do not suggest code. \
    Finally suggest a concrete list of the actions that Coder should take to improve the codes based on the critique above.",
    llm_config=llm_config_assistant_critic,
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, critic], messages=[], max_round=15)
groupchatmanager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_groupchatmanager)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

user_proxy.initiate_chat(
    groupchatmanager,
    message="Read data from './sales_data_analytics.csv' and accomplish the analysis as following steps. \
    Step 1: Calculate the total sales for each 'store'. \
    Step 2: Then plot a visualization that shows relationship between CompetitionDistance and Total Sales of each store. \
    Step 3: At last, export the plot as an PNG, remember to print the fields in a dataset before visualizing it.",
)

HumanAdmin (to chat_manager):

Read data from './sales_data_analytics.csv' and accomplish the analysis as following steps.     Step 1: Calculate the total sales for each 'store'.     Step 2: Then plot a visualization that shows relationship between CompetitionDistance and Total Sales of each store.     Step 3: At last, export the plot as an PNG, remember to print the fields in a dataset before visualizing it.

--------------------------------------------------------------------------------
Coder (to chat_manager):

Sure, I can help you with that. Let's start by reading the data from the CSV file and processing it to perform the steps you outlined. I will use Python for this task. Let's start with step 1:

```python
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Read data from the CSV file
data = pd.read_csv('./sales_data_analytics.csv')

# Calculate total sales for each store
total_sales_per_store = data.groupby('Store')['Sales'].sum().reset_index()

# Display the calcu

ChatResult(chat_id=None, chat_history=[{'content': "Read data from './sales_data_analytics.csv' and accomplish the analysis as following steps.     Step 1: Calculate the total sales for each 'store'.     Step 2: Then plot a visualization that shows relationship between CompetitionDistance and Total Sales of each store.     Step 3: At last, export the plot as an PNG, remember to print the fields in a dataset before visualizing it.", 'role': 'assistant'}, {'content': "Sure, I can help you with that. Let's start by reading the data from the CSV file and processing it to perform the steps you outlined. I will use Python for this task. Let's start with step 1:\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Step 1: Read data from the CSV file\ndata = pd.read_csv('./sales_data_analytics.csv')\n\n# Calculate total sales for each store\ntotal_sales_per_store = data.groupby('Store')['Sales'].sum().reset_index()\n\n# Display the calculated total sales per store\nprint(tot